# Tarea 10

_Tarea 10_ de _Benjamín Rivera_
para el curso de __Métodos Numéricos__
impartido por _Joaquín Peña Acevedo_.

Fecha limite de entrega __...__.

In [1]:
import sys
import seaborn as sns
import scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_triangular # Para backward y forward substitution

NOTEBOOK = True

## Ejercicio 1

Programar el m ́etodo de Romberg para estimar el valor de la integral deuna funcion real $f(x)$ en un intervalo dado.

### Regla del trapecio

Escribir el codigo de la funcion que aplica la regla de trapecio de manera recursiva. La funcion __recibe__ como argumentos 
 - el apuntador a la funcion $f(x)$, 
 - un entero $i$ y 
 - el valor $R(i-1,0)$.
 
> tambien pasare la tupla del limite $(a,b)$ que creo son necesarios
 
Si $i=0$, calcular 
    $$ R(i,0) = \frac{b-a}{2}[f(a) +f(b)] $$
(sin importar el valor de $R(i-1,0)$).

Para $i\geq0$, calcular $h=\frac{b-a}{2^i}$ y 
    $$ R(i,0) = \frac{1}{2} R(i-1,0) + h \sum^{2^{i−1}}_{k=1} f(a+ (2k−1)h) $$
    
La funcion __debe devolver__ el valor de $R(i,0)$.

In [2]:
def romberg_iterativo(f, i, lim):
    """ Funcion para ejecutar el metodo de Romberg iterativo. 
    
    Esta funcion trata de ejecutar el metodo de Romberg iterativo 
    con un acercamiento iterativo. Regresara el valor calculado, 
    con la condicion de termino i == 0 donde no se llamara a si 
    misma de nuevo.
    
    Para que esta funcion sea un poco mas "pythonica" no pediremos 
    el valor previo del metodo y lo calcularemos de manera interna
    con el enfoque recursivo. 
    Claramente este no es el metodo que se explica en la tarea,
    pero el otro se me revolvio y este funciona (aunque algo lento)
    
    Input:
        f := funcion para aproximar integral
        i := iteracion del metodo
        lim := tupla de limites de la evaluacion
    
    Regresa el valor de R(i,0), un flotante.
    """
    
    try:
        a, b = lim
    except TypeError:
        raise Exception("Err: lim debe contener valores (a,b)")

    if i == 0:
        return ((b-a)/2)*(f(a) + f(b))
    else:
        h = (b-a)/(2**i)
        r_im1 = romberg_iterativo(f, i-1, lim)
        
        suma = 0
        for k in range(1, 2**(i-1)+1):
            suma += f(a + (2*k - 1)*h)
        
        return 0.5*r_im1 + h*suma

In [3]:
def regla_trapecio_iterativa(f, i, r_im1, lim):
    """ Funcion para ejecutar la regla del trapecio. 
    
    Esta funcion trata de ejecutar la regla del trapecio mediante
    un acercamiento iterativo. Regresara el valor calculado, con
    la condicion de termino i == 0 donde no se llamara a si misma
    de nuevo.
    
    Este es mi intento de implementacion de la funcion solicitada
    en la tarea. Algo que no se solicita en la tarea pero que si
    se pide en la funcion, son los limites de integracion.
    
    Input:
        f := funcion para aproximar integral
        i := iteracion del metodo
        rim1 := Evaluacion de la regla en la iteracion anterior.
        lim := tupla de limites de la evaluacion
    
    Regresa el valor de R(i,0), un flotante.
    """
    try:
        a, b = lim
    except TypeError:
        raise Exception("Err: lim debe contener valores (a,b)")

    if i == 0:
        return ((b-a)/2)*(f(a) + f(b))
    else:
        h = (b-a)/(2**i)
        suma = 0
        for k in range(1, 2**(i-1)+1):
            suma += f(a + (2*k - 1)*h)
        
        return 0.5*r_im1 + h*suma


### Metodo Romberg

Escribir una funci ́on que implementa el metodo de Romberg. La funcion recibe como argumentos 
 - el apuntador a la funcion $f(x)$, 
 - un entero $n$, 
 - los lımites de integracion $a$ y $b$.  
 
La funcion crea una matriz $R$ de tamano $(n+ 1)\times(n+ 1)$ y la inicializa con ceros. Luego calcula los valores $R(i,0)$ usando la funci ́on del punto anterior para $i= 0,1,...,n$. Luego  calcula  los  valores $R(i,j)$  usando  extrapolacion  de  Richardson  para $j=1,2,...,n$:
    $$ R(i,j) = R(i,j-1) + \frac{1}{4^j-1} [R(i,j-1) - R(i-1,j-1)] $$

La funci ́on devuelve la matriz $R$.

In [4]:
def romberg_matricial(f, n, lim):
    """Funcion para ejecutar el metodo de Romberg iterativo. 
    
    Esta funcion trata de ejecutar el metodo de Romberg con la 
    ayuda de una matriz para guardar las iteraciones del mismo. 
    Regresara la matriz calculada de las iteraciones.
    
    Input:
        f := funcion para aproximar integral
        n := profundidad de iteracion del metodo
        lim := tupla de limites de la evaluacion
    
    Regresara la matriz calculada con las iteraciones.
    """
    # Preparacion de datos
    try:
        a, b = lim
    except TypeError:
        raise Exception("Err: lim debe contener valores (a,b)")
    n += 1
    # init matriz
    matriz_romberg = np.zeros((n,n))
    
    for i in range(n):
        matriz_romberg[i, 0] = regla_trapecio_iterativa(f, i, 
                                                        matriz_romberg[i-1, 0], 
                                                        lim)
        
    for j in range(1, n):
        for i in range(n):
            matriz_romberg[i,j] = (matriz_romberg[i, j-1] 
                                   + 1/(4**j - 1)
                                   * (matriz_romberg[i, j-1] 
                                      - matriz_romberg[i-1, j-1])
                                  )
    
    return matriz_romberg


### Funciones

Programar las siguientes funciones y sus primitivas

$$ f_1 = 6x^5 - 4x^3 - 2x + 1 \quad F(x) = x^6 - x^4 - x^2 + x$$
$$ f_2 = \sin 2\pi x \quad F_2$$

In [5]:
f1 = lambda x: 6*x**5 - 4*x**3 - 2*x + 1
F1 = lambda x: x**6 - x**4 - x**2 + x

f2 = lambda x: np.sin(x*2*np.pi)
F2 = lambda x: -np.cos(2*np.pi-x)/(2*np.pi)

### UI

Si quiere ingresar los parametros durante ejecucion (que sean preguntados), ponga la variable `ASK` a _True_

In [6]:
def Ejercicio1(lim, N):
    print(f"Limites: {lim}")
    print(f"Divisiones: {N}")
    
    funciones = (('Polinomio', f1, F1), ('Trigonometrico', f2, F2))
    
    for nombre, f, F in funciones:
        temp_mat = romberg_matricial(f, N, lim)
        print("\n" + nombre)
        print("Matriz")
        print(temp_mat)
        print("\n" + "- "*25)
        print(f"Romberg: {temp_mat[N,N]}")
        temp = F(lim[1]) - F(lim[0])
        print(f"Verdadero: {temp}")
        print(f"Relativo: {abs(temp - temp_mat[N,N])/abs(temp)}")
        print("_"*50 + "\n\n")


In [7]:
ASK = False

valores = [(0.001, 2.5, 3), (0.001, 2.5, 6), 
        (0.000001, 2.5, 3), (0.000001, 2.5, 6)]

if ASK:
    a = input()
    b = input()
    N = input()
    Ejercicio1((a, b), N)
else:
    for a, b, N in valores:
        Ejercicio1((a, b), N)


Limites: (0.001, 2.5)
Divisiones: 3

Polinomio
Matriz
[[650.28415725 796.97333521 836.67514791 846.76003715]
 [336.41796462 231.79590041 194.11740476 183.9180755 ]
 [236.52805946 203.2314244  201.327126   201.44156602]
 [210.21662335 201.44614465 201.327126   201.327126  ]]

- - - - - - - - - - - - - - - - - - - - - - - - - 
Romberg: 201.327126000001
Verdadero: 201.327126000001
Relativo: 0.0
__________________________________________________



Trigonometrico
Matriz
[[ 0.00785079 -0.05916486 -0.0874157  -0.09578277]
 [ 1.25341923  1.66860871  1.78379361  1.81349535]
 [-0.2582027  -0.76207667 -0.92412236 -0.96710516]
 [ 0.20889772  0.36459786  0.4397095   0.46135762]]

- - - - - - - - - - - - - - - - - - - - - - - - - 
Romberg: 0.46135762428031185
Verdadero: 0.28666083005523796
Relativo: 0.6094198296691278
__________________________________________________


Limites: (0.001, 2.5)
Divisiones: 6

Polinomio
Matriz
[[650.28415725 799.89004898 839.79424191 849.92864057 852.47217592
  853.108

No siempre mejora, el aumento de las iteraciones no asegura un aumento definitivo; el claro ejemplo esta en el primero punto, donde con 3 iteraciones no hay error, pero para 6 si se desbia. Fuera de este, con la otra funcion si hay una mejora porcentual considerable.